In [23]:
import pandas as pd
import numpy as np
import gensim.downloader as api
import ast
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential, layers
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping

In [24]:
file_path = '~/code/TechLah/RevuSum/data/processed_kswdf.csv'
hotelreviews = pd.read_csv(file_path)
df = hotelreviews.copy()

In [25]:
df = df.groupby('Label', group_keys=False).apply(lambda x: x.sample(10000))
df['Label'].value_counts()

Label
0    10000
1    10000
Name: count, dtype: int64

In [26]:
X = df['Review']
X = [ast.literal_eval(string) for string in X]
y = df['Label']

X_train, X_test, y_train, y_test = train_test_split(X,y, shuffle=True,test_size=0.3,random_state=1)

## Tried using pre-trained model (Word2Vec) instead

In [27]:
print(list(api.info()['models'].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [28]:
word2vec_model = api.load("glove-wiki-gigaword-50")

In [29]:
# Function to convert a sentence (list of words) into a matrix representing the words in the embedding space
def embed_sentence_with_TF(word2vec, sentence):
    embedded_sentence = []
    for word in sentence:
        if word in word2vec:
            embedded_sentence.append(word2vec[word])
        
    return np.array(embedded_sentence)

# Function that converts a list of sentences into a list of matrices
def embedding(word2vec, sentences):
    embed = []
    
    for sentence in sentences:
        embedded_sentence = embed_sentence_with_TF(word2vec, sentence)
        embed.append(embedded_sentence)
        
    return embed

In [30]:
# Embed the training and test sentences
X_train_embed = embedding(word2vec_model, X_train)
X_test_embed = embedding(word2vec_model, X_test)


In [31]:
max_train_length = max(len(lst) for lst in X_train_embed)
max_test_length = max(len(lst) for lst in X_test_embed)
maxlen = max(max_train_length,max_test_length)

In [32]:
# Pad the training and test embedded sentences
X_train_emb_pad = pad_sequences(X_train_embed, dtype='float16', padding='post', maxlen=maxlen)
X_test_emb_pad = pad_sequences(X_test_embed, dtype='float16', padding='post', maxlen=maxlen)

In [33]:
def init_model():
    model = Sequential()
    model.add(layers.Masking())
    model.add(layers.LSTM(20, activation='tanh'))
    model.add(layers.Dense(15, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy',
                  optimizer='rmsprop',
                  metrics=['accuracy'])
    
    return model

2023-06-22 14:20:24.725587: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-22 14:20:24.725917: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [34]:
es = EarlyStopping(patience=5, restore_best_weights=True)

model = init_model()

model.fit(X_train_emb_pad, y_train, 
          batch_size = 32,
          epochs=10,
          validation_split=0.3,
          callbacks=[es]
         )

Epoch 1/10
307/307 [==============================] - 31s 93ms/step - loss: 0.4396 - accuracy: 0.7946 - val_loss: 0.2854 - val_accuracy: 0.8886
Epoch 2/10
307/307 [==============================] - 27s 88ms/step - loss: 0.2710 - accuracy: 0.8917 - val_loss: 0.2447 - val_accuracy: 0.9021
Epoch 3/10
307/307 [==============================] - 26s 86ms/step - loss: 0.2381 - accuracy: 0.9066 - val_loss: 0.2163 - val_accuracy: 0.9171
Epoch 4/10
307/307 [==============================] - 27s 89ms/step - loss: 0.2144 - accuracy: 0.9165 - val_loss: 0.3039 - val_accuracy: 0.8690
Epoch 5/10
307/307 [==============================] - 27s 88ms/step - loss: 0.2011 - accuracy: 0.9208 - val_loss: 0.2050 - val_accuracy: 0.9207
Epoch 6/10
307/307 [==============================] - 27s 89ms/step - loss: 0.1931 - accuracy: 0.9260 - val_loss: 0.1937 - val_accuracy: 0.9264
Epoch 7/10
307/307 [==============================] - 27s 87ms/step - loss: 0.1820 - accuracy: 0.9301 - val_loss: 0.1852 - val_accuracy:

In [35]:
res = model.evaluate(X_test_emb_pad, y_test, verbose=0)

print(f'The accuracy evaluated on the test set is of {res[1]*100:.3f}%')

The accuracy evaluated on the test set is of 90.583%
